In [1]:
import os
import re
import warnings
from copy import deepcopy

import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import RobustScaler


import matplotlib.pyplot as plt

plt.style.use('Estilo/toutl.mplstyle')
from style import *


In [2]:
df = pd.read_csv('Datos/examen3.csv')

In [3]:
columnas_analizar = df.select_dtypes(include=['int64', 'float64']).drop(columns=['clientnum', 'client_stayed'], errors='ignore').columns.tolist()

def generar_reporte_outliers(df, columnas):
    print("\n" + "="*92)
    print(f"{'REPORTE DE OUTLIERS - ANÁLISIS COMPLETO':^92}")
    print("="*92)
    print(f"{'Columna':<25} {'Tipo':<10} {'Outliers':>10} {'%':>8} {'L. Inferior':>15} {'L. Superior':>15}")
    print("-"*92)
    
    for col in columnas:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lim_inf = Q1 - 1.5 * IQR
        lim_sup = Q3 + 1.5 * IQR
        outliers = df[(df[col] < lim_inf) | (df[col] > lim_sup)]
        n_outliers = len(outliers)
        pct_outliers = (n_outliers/len(df))*100
        
        str_lim_inf = f"{lim_inf:,.2f}" if isinstance(lim_inf, float) else f"{int(lim_inf):,}"
        str_lim_sup = f"{lim_sup:,.2f}" if isinstance(lim_sup, float) else f"{int(lim_sup):,}"
        
        print(f"{col:<25} {str(df[col].dtype):<10} {n_outliers:>10,} {pct_outliers:>7.2f}% {str_lim_inf:>15} {str_lim_sup:>15}")

    print("="*92)

generar_reporte_outliers(df, columnas_analizar)


                          REPORTE DE OUTLIERS - ANÁLISIS COMPLETO                           
Columna                   Tipo         Outliers        %     L. Inferior     L. Superior
--------------------------------------------------------------------------------------------
customer_age              int64               2    0.02%           24.50           68.50
dependent_count           int64               0    0.00%           -2.00            6.00
months_on_book            int64             386    3.81%           17.50           53.50
total_relationship_count  int64               0    0.00%            0.00            8.00
months_inactive_12_mon    int64             331    3.27%            0.50            4.50
contacts_count_12_mon     int64             629    6.21%            0.50            4.50
credit_limit              float64           984    9.72%      -10,213.75       23,836.25
total_revolving_bal       int64               0    0.00%       -1,778.50        3,921.50
total_trans_

In [4]:
def generar_reporte_outliers_por_grupo(df, columnas, grupo_col='client_stayed'):
    col_width = 22
    type_width = 10
    outliers_width = 10
    pct_width = 8
    limit_width = 15
    
    for grupo in sorted(df[grupo_col].unique()):
        print(f"\n{'ANÁLISIS PARA CLIENT_STAYED =':<20} {grupo}")
        print("-"*100)
        print(f"{'Columna':<{col_width}} {'Tipo':<{type_width}} {'Outliers':>{outliers_width}} "
              f"{'%':>{pct_width}} {'L. Inferior':>{limit_width}} {'L. Superior':>{limit_width}}")
        print("-"*100)
        
        grupo_df = df[df[grupo_col] == grupo]
        
        for col in columnas:
            Q1 = grupo_df[col].quantile(0.25)
            Q3 = grupo_df[col].quantile(0.75)
            IQR = Q3 - Q1
            lim_inf = Q1 - 1.5 * IQR
            lim_sup = Q3 + 1.5 * IQR
            
            outliers = grupo_df[(grupo_df[col] < lim_inf) | (grupo_df[col] > lim_sup)]
            n_outliers = len(outliers)
            pct_outliers = (n_outliers/len(grupo_df))*100
            
            is_float = isinstance(grupo_df[col].iloc[0], float)
            fmt_lim = "{:,.2f}" if is_float else "{:,}"
            
            print(f"{col:<{col_width}} {str(grupo_df[col].dtype):<{type_width}} "
                  f"{n_outliers:>{outliers_width},} {pct_outliers:>{pct_width-1}.2f}% "
                  f"{fmt_lim.format(lim_inf):>{limit_width}} {fmt_lim.format(lim_sup):>{limit_width}}")
    
    print("="*100)

generar_reporte_outliers_por_grupo(df, columnas_analizar)


ANÁLISIS PARA CLIENT_STAYED = 0
----------------------------------------------------------------------------------------------------
Columna                Tipo         Outliers        %     L. Inferior     L. Superior
----------------------------------------------------------------------------------------------------
customer_age           int64               0    0.00%            24.5            68.5
dependent_count        int64             199   12.23%             0.5             4.5
months_on_book         int64              77    4.73%            20.0            52.0
total_relationship_count int64               0    0.00%            -2.5             9.5
months_inactive_12_mon int64              66    4.06%             0.5             4.5
contacts_count_12_mon  int64               0    0.00%            -1.0             7.0
credit_limit           float64           173   10.63%       -9,615.25       21,662.75
total_revolving_bal    int64               0    0.00%       -1,955.25      

In [5]:

def agregar_columna_vip(df, columna_credito='credit_limit', grupo_col='client_stayed', 
                        metodo='IQR', umbral=1.5, inplace=False):
    
    if not inplace:
        df = df.copy()
    
    
    df['vip'] = 0
    
    for grupo in df[grupo_col].unique():
        mascara_grupo = df[grupo_col] == grupo
        datos_credito = df.loc[mascara_grupo, columna_credito]
        
        
        if metodo == 'IQR':
            Q1 = datos_credito.quantile(0.25)
            Q3 = datos_credito.quantile(0.75)
            IQR = Q3 - Q1
            limite_superior = Q3 + umbral * IQR
        elif metodo == 'zscore':
            mediana = datos_credito.median()
            mad = np.median(np.abs(datos_credito - mediana))
            limite_superior = mediana + umbral * mad / 0.6745
        else:
            raise ValueError("Método debe ser 'IQR' o 'zscore'")
        
      
        df.loc[(mascara_grupo) & (df[columna_credito] > limite_superior), 'vip'] = 1
    
    
    total_vips = df['vip'].sum()
    print(f"Total clientes VIP identificados: {total_vips}")
    print("\nDistribución de VIPs por grupo:")
    print(pd.crosstab(df[grupo_col], df['vip'], margins=True))
    
    if not inplace:
        return df

In [6]:
df_nuevo = agregar_columna_vip(
    df=df,
    columna_credito='credit_limit',
    grupo_col='client_stayed',
    metodo='IQR',
    umbral=1.5
)
df

Total clientes VIP identificados: 967

Distribución de VIPs por grupo:
vip               0    1    All
client_stayed                  
0              1454  173   1627
1              7706  794   8500
All            9160  967  10127


,clientnum,client_stayed,customer_age,gender,dependent_count,education_level,marital_status,income_category,card_category,months_on_book,total_relationship_count,months_inactive_12_mon,contacts_count_12_mon,credit_limit,total_revolving_bal,total_trans_amt,total_trans_ct
0,768805383,1,45,M,3,High School,Married,5833.333333333333,Blue,39,5,1,3,12691.0,777,1144,42
1,818770008,1,49,F,5,Graduate,Single,1666.6666666666667,Blue,44,6,1,2,8256.0,864,1291,33
2,713982108,1,51,M,3,Graduate,Married,8333.333333333334,Blue,36,4,1,0,3418.0,0,1887,20
3,769911858,1,40,F,4,High School,Unknown,1666.6666666666667,Blue,34,3,4,1,3313.0,2517,1171,20
4,709106358,1,40,M,3,Uneducated,Married,5833.333333333333,Blue,21,5,1,0,4716.0,0,816,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10122,772366833,1,50,M,2,Graduate,Single,4166.666666666667,Blue,40,3,2,3,4003.0,1851,15476,117
10123,710638233,0,41,M,2,Unknown,Divorced,4166.666666666667,Blue,25,4,2,3,4277.0,2186,8764,69
10124,716506083,0,44,F,1,High School,Married,1666.6666666666667,Blue,36,5,3,4,5409.0,0,10291,60
10125,717406983,0,30,M,2,Graduate,Unknown,4166.666666666667,Blue,36,4,3,3,5281.0,0,8395,62


In [7]:
def sustituir_outliers_por_grupo(df, columnas_a_tratar, grupo_col='client_stayed', 
                                metodo='IQR', umbral=1.5, estrategia='limite'):
    
    df_tratado = df.copy(deep=True)
    

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        
        for grupo in df[grupo_col].unique():
            mascara_grupo = df[grupo_col] == grupo
            
            for columna in columnas_a_tratar:
                if columna not in df.columns:
                    continue
                    
                datos_grupo = df.loc[mascara_grupo, columna]
                
             
                if metodo == 'IQR':
                    Q1 = datos_grupo.quantile(0.25)
                    Q3 = datos_grupo.quantile(0.75)
                    IQR = Q3 - Q1
                    limite_inferior = Q1 - umbral * IQR
                    limite_superior = Q3 + umbral * IQR
                elif metodo == 'zscore':
                    mediana = datos_grupo.median()
                    mad = np.median(np.abs(datos_grupo - mediana))
                    limite_inferior = mediana - umbral * mad / 0.6745
                    limite_superior = mediana + umbral * mad / 0.6745
                else:
                    raise ValueError("Método no reconocido. Use 'IQR' o 'zscore'")
                
            
                es_outlier = ((datos_grupo < limite_inferior) | 
                             (datos_grupo > limite_superior))
                
                if not es_outlier.any():  
                    continue
                    
         
                if estrategia == 'limite':
                    valores_corregidos = np.clip(
                        df.loc[mascara_grupo & es_outlier, columna], 
                        limite_inferior, 
                        limite_superior
                    )
                elif estrategia == 'mediana':
                    valores_corregidos = datos_grupo.median()
                elif estrategia == 'media_recortada':
                    mascara_no_outliers = ~es_outlier & mascara_grupo
                    valores_corregidos = df.loc[mascara_no_outliers, columna].mean()
                else:
                    raise ValueError("Estrategia no reconocida. Use 'limite', 'mediana' o 'media_recortada'")
                
                df_tratado.loc[mascara_grupo & es_outlier, columna] = valores_corregidos.astype(df[columna].dtype)
    
    return df_tratado

In [8]:
df['credit_limit'] = df['credit_limit'].astype('int64')

columnas_a_tratar = df.select_dtypes(include=['int64']).drop(
    columns=['clientnum', 'client_stayed', 'customer_age','vip'], 
    errors='ignore'
).columns.tolist()

In [9]:
df = sustituir_outliers_por_grupo(
    df=df,
    columnas_a_tratar=columnas_a_tratar,
    grupo_col='client_stayed',
    metodo='IQR',
    umbral=1.5,
    estrategia='limite'  # Corregido: era 'estrategia' en tu llamada original
)


In [11]:
generar_reporte_outliers_por_grupo(df, columnas_a_tratar)


ANÁLISIS PARA CLIENT_STAYED = 0
----------------------------------------------------------------------------------------------------
Columna                Tipo         Outliers        %     L. Inferior     L. Superior
----------------------------------------------------------------------------------------------------
dependent_count        int64             135    8.30%             0.5             4.5
months_on_book         int64               0    0.00%            20.0            52.0
total_relationship_count int64               0    0.00%            -2.5             9.5
months_inactive_12_mon int64              15    0.92%             0.5             4.5
contacts_count_12_mon  int64               0    0.00%            -1.0             7.0
credit_limit           int64               0    0.00%       -9,615.25       21,662.75
total_revolving_bal    int64               0    0.00%       -1,955.25        3,258.75
total_trans_amt        int64               7    0.43%          600.75      